In [1]:
import numpy as np

# Implementacja NSGA-II

In [31]:
class NSGAII:
    def __init__(self, chromosome_length, population_size, objectives_number,
                 initial_population, initial_objective_values):
        self.chromosome_length = chromosome_length
        self.population_size = population_size
        self.objectives_number = objectives_number
        if type(initial_population) is not np.ndarray:
            raise ValueError("initial population must be a numpy array")
        if initial_population.shape != (self.population_size, self.chromosome_length):
            raise ValueError("initial population must be of shape (population_size, chromosome_length)")
        self.current_population = initial_population
        if type(initial_objective_values) is not np.ndarray:
            raise ValueError("initial objective values must be a numpy array")
        if initial_objective_values.shape != (self.population_size, self.objectives_number):
            raise ValueError("initial population values must be of shape (population_size, objectives_number)")
        self.current_objective_values = initial_objective_values
        self.current_pareto_frontier = np.empty(self.population_size)
        self.current_crowding_distances = np.empty(self.population_size)
        self.replace_population(np.array([]), np.array([]))


    def get_population(self):
        return self.current_population


    def non_dominated_sort(self, objective_values):

        def dominates(vals1, vals2):
            return np.all(vals1 >= vals2) and np.any(vals1 > vals2)

        size = objective_values.shape[0]
        pareto_frontier = np.empty(size, dtype=int)
        current_frontier_number = 1
        current_frontier_members = []
        # number of solutions dominating p
        dominating_count = np.zeros(size, dtype=int)
        # set of solutions dominated by p
        dominated = [None] * size 

        for p in range(size):
            dominated[p] = []
            for q in range(size):
                if dominates(objective_values[p], objective_values[q]):
                    dominated[p].append(q)
                elif dominates(objective_values[q], objective_values[p]):
                    dominating_count[p] += 1
            if dominating_count[p] == 0:
                pareto_frontier[p] = current_frontier_number
                current_frontier_members.append(p)

        while current_frontier_members != []:
            current_frontier_number += 1
            new_frontier_members = []
            for p in current_frontier_members:
                for q in dominated[p]:
                    dominating_count[q] -= 1
                    if dominating_count[q] == 0:
                        pareto_frontier[q] = current_frontier_number
                        new_frontier_members.append(q)
            current_frontier_members = new_frontier_members
        
        return pareto_frontier


    def assign_crowding_distance(self, objective_values):
        size = objective_values.shape[0]
        distance = np.zeros(size)
        for current_objective in range(self.objectives_number):
            current_objective_values = objective_values[:, current_objective]
            sorted_indices = np.argsort(current_objective_values)
            current_objective_values_diff = current_objective_values[sorted_indices[-1]] - current_objective_values[sorted_indices[0]]
            if current_objective_values_diff == 0:
                continue
            distance[sorted_indices[0]], distance[sorted_indices[-1]] = np.inf, np.inf
            for i in range(1, size-1):
                distance[sorted_indices[i]] += ((current_objective_values[sorted_indices[i+1]] - current_objective_values[sorted_indices[i-1]]) /
                                                current_objective_values_diff)
        return distances


    def generate_children(self, objective_values, number_of_offspring = self.population_size):
        # TODO
        # parent selection
        # recombination
        # mutation
        pass


    def replace_population(self, children_population, children_objective_values):

        combined_population = np.vstack(self.current_population, children_population)
        combined_objective_values = np.vstack(self.current_objective_values, children_objective_values)
        combined_pareto_frontier = non_dominated_sort(combined_objective_values)

        positions = np.argsort(combined_pareto_frontier)

        combined_population = combined_population[positions]
        combined_objective_values = combined_objective_values[positions]
        combined_pareto_frontier = combined_pareto_frontier[positions]

        # assign crowding distance and select subset of last frontier
        found_population_size = 0
        current_frontier_number = 1
        current_frontier_lb, current_frontier_rb = (np.searchsorted(combined_pareto_frontier, current_frontier_number),
                                                    np.searchsorted(combined_pareto_frontier, current_frontier_number, side='right'))
        current_frontier_size = current_frontier_rb - current_frontier_lb
        while found_population_size + current_frontier_size <= self.population_size:
            current_frontier_distances = self.assign_crowding_distance(combined_objective_values[current_frontier_lb : current_frontier_rb])
            self.current_crowding_distances[current_frontier_lb : current_frontier_rb] = current_frontier_distances
            current_frontier_number += 1
            current_frontier_lb, current_frontier_rb = (np.searchsorted(combined_pareto_frontier, current_frontier_number),
                                                        np.searchsorted(combined_pareto_frontier, current_frontier_number, side='right'))
            current_frontier_size = current_frontier_rb - current_frontier_lb

        current_frontier_distances = self.assign_crowding_distance(combined_objective_values[current_frontier_lb : current_frontier_rb])
        last_frontier_taken_indices = np.argsort(current_frontier_distances)[::-1][:self.population_size - found_population_size]

        combined_population[current_frontier_lb : self.population_size] = combined_population[last_frontier_taken_indices]
        combined_objective_values[current_frontier_lb : self.population_size] = combined_objective_values[last_frontier_taken_indices]
        self.current_crowding_distances[current_frontier_lb : self.population_size] = current_frontier_distances[last_frontier_taken_indices]

        self.current_population = combined_population[:self.population_size]
        self.current_objective_values = combined_objective_values[:self.population_size]
        self.current_pareto_frontier = combined_pareto_frontier[:self.population_size]

        return True

NameError: ignored

In [ ]:
l = [1]
l == []

False

In [30]:
np.argsort(np.arange(5))[::-1][:1]

array([4])

In [12]:
np.searchsorted(np.array([1,2,2,3,3,3,4,5,5]), 3, side='right')

6

In [19]:
np.zeros(1) / 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([nan])

In [23]:
np.array([[1, 2], [3, 4], [5, 6]])[:, 0]

array([1, 3, 5])